In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

### Test Local LangGraph Agent

In [ ]:
from langgraph_sdk import get_client

url = "https://leetmock-ts-fa225f46565756e7b0567441810f232f.default.us.langgraph.app"
client = get_client(url=url)

# Using the graph deployed with the name "agent"
assistant_id = "code-mock-staged-v1"

# create thread
thread = await client.threads.create()

print(thread)

In [2]:
openai_assistant = await client.assistants.create(graph_id=assistant_id)

In [ ]:
openai_assistant

In [3]:
from agent_graph.code_mock_staged_v1.graph import create_compiled_graph
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

# graph = create_graph().compile()
graph = create_compiled_graph()
config = { "configurable": {"thread_id": "1", "session_id1": "abc"} }

In [ ]:
async for chunk in graph.astream(
    input={
        "messages": ["Hi"],
        "event": "reminder",
        # "trigger": True,
    },
    config=config,
    stream_mode=["values"],
):
    print(chunk)

async for chunk in graph.astream(
    input={
        "messages": ["Hi"],
        # "event": "user_message"
    },
    config=config,
    stream_mode=["values"],
):
    print(chunk)

In [ ]:
from langchain_core.load.load import load, loads
from langchain_core.load.dump import dumpd, dumps
from langgraph.types import StateSnapshot

state = graph.get_state(config=config)
state

In [ ]:
dumped_state = dumps(state)
dumped_state

In [ ]:
loaded_state = loads(dumped_state, valid_namespaces=["agent_graph"])
loaded_state_snapshot = StateSnapshot(*loaded_state)
loaded_state_snapshot

In [ ]:
loaded_state_snapshot == state, dumpd(loaded_state_snapshot) == dumpd(state)

In [ ]:
loaded_state_snapshot.values == state.values

In [14]:
history = list(graph.get_state_history(config=config))

In [ ]:
dumped_history = dumps(history)
dumped_history

In [ ]:
loaded_history = loads(dumped_history, valid_namespaces=["agent_graph"])
loaded_history_snapshot = [StateSnapshot(*h) for h in loaded_history]
loaded_history_snapshot

In [ ]:
loaded_history_snapshot == history, dumpd(loaded_history_snapshot) == dumpd(history)

In [ ]:
dumped_state_byte = dumped_state.encode()
dumped_history_byte = dumped_history.encode()
print(f"State: {len(dumped_state_byte) / 1024:.2f} KB")
print(f"State History: {len(dumped_history_byte) / 1024:.2f} KB")